In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

logging.langsmith("allforone")

In [ ]:
import json

with open(
    "src/agents/analysis/run_test/location_insight_agent.ipynb", "r", encoding="utf-8"
) as f:
    notebook = json.load(f)

notebook["metadata"]["kernelspec"]["display_name"] = "RAG_Commander"
notebook["metadata"]["kernelspec"]["name"] = "python3"

with open(
    "src/agents/analysis/run_test/location_insight_agent.ipynb", "w", encoding="utf-8"
) as f:
    json.dump(notebook, f, ensure_ascii=False, indent=1)

print("커널 설정을 수정했습니다.")

In [1]:
from langgraph.graph import StateGraph, START, END
from agents.state.analysis_state import LocationInsightState
from agents.state.start_state import StartInput
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from utils.util import get_today_str
from utils.llm import LLMProfile
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from prompts import PromptManager, PromptType
from langgraph.prebuilt import ToolNode
from tools.kostat_api import get_move_population, system_prompt
import json
import asyncio


@tool(parse_docstring=False)
def think_tool(reflection: str) -> str:
    """
    [역할]
    당신은 입지/호재 정리 전문가의 내부 반성·점검(Reflection) 담당자입니다.
    최종 보고서에 들어갈 본문(Markdown)을 쓰기 직전에, 데이터 품질·핵심 수치·리스크·보고서용 한 줄 메시지를 짧고 구조적으로 요약해 think_tool에 기록합니다. 이 반성문은 내부용이며, 최종 보고서에 직접 노출되지 않습니다.

    [언제 호출할 것인지]
    - 데이터 수집/정제 → 핵심 수치 산출 → 시계열 해석을 마친 직후 1회 호출(필수)
    - 추가 데이터로 최신 데이터로 바뀌면 갱신 시마다 1회 재호출(선택)

    [강력 지시]
    - 해당 지역에 관련된 내용만 기록
    - 허상 가정,출처 수치 금지
    - 다음 단계(보고서 에이전트)가 바로 쓸 수 있는 한 줄 핵심 메시지 포함

    [나쁜 예]
    - “경제가 좋아진듯함. 분위기 좋음.”(수치·기간·단위·근거 없음)
    - “인근 해운대의 입지지는 이렇다~”(대상 지역 외 서술)
    - “향후 집값 상승 확실.”(근거 없는 단정)

    [검증 체크리스트]
    - 정량 수치가 어긋난 것 이 있는가?
    - GPT가 시계열 판단하기에 좋은 형식으로 되어있는가?
    - 잘못된 내용은 없는가?
    """
    return f"Reflection recorded: {reflection}"


output_key = LocationInsightState.KEY.location_insight_output
start_input_key = LocationInsightState.KEY.start_input
messages_key = LocationInsightState.KEY.messages
target_area_key = StartInput.KEY.target_area
main_type_key = StartInput.KEY.main_type
total_units_key = StartInput.KEY.total_units
web_context_key = LocationInsightState.KEY.web_context
kakao_api_distance_context_key = LocationInsightState.KEY.kakao_api_distance_context
gemini_search_key = LocationInsightState.KEY.gemini_search
perplexity_search_key = LocationInsightState.KEY.perplexity_search


llm = LLMProfile.analysis_llm()
tool_list = [think_tool]
llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

from tools.gemini_search_tool import gemini_search


def gemini_search_tool(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    main_type = start_input[main_type_key]
    total_units = start_input[total_units_key]

    prompt = f"""
    <CONTEXT>
    주소: {target_area}
    규모: {total_units}세대
    타입: {main_type}
    </CONTEXT>
    <GOAL>
    <CONTEXT> 주변 분양호재를 <OUTPUT>을 참조해서 json 형식으로 출력해주세요
    </GOAL>
    <OUTPUT>
    {{
      "분양호재": [
        {{
          "name": "",
          "location": "",
          "description": "",
          "status": ""
        }}
      ]
    }}
    </OUTPUT>
    """
    result = gemini_search(prompt)

    return {gemini_search_key: result}


from tools.perplexity_search_tool import perplexity_search


def perplexity_search_tool(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    main_type = start_input[main_type_key]
    total_units = start_input[total_units_key]
    date = get_today_str()

    prompt = f"""
    <CONTEXT>
    주소: {target_area}
    규모: {total_units}세대
    타입: {main_type}
    일시: {date}
    해당 목표일에 주소에서 아파트 분양을 계획하고 있습니다.
    </CONTEXT>

    <GOAL>
    - <CONTEXT>에 나와 있는 스펙으로 해당 사업지에 맞는 핵심 데이터 선별을 위해해 부동산과 관련된 해당지역에 관한 특징을 추려서 출력해주세요.
    - <CONTEXT> 주변 분양호재를 <OUTPUT>을 참조해서 json 형식으로 출력해주세요
    </GOAL>
    <OUTPUT>
    {{
      "해당지역 특징": []
      "분양호재": [
        {{
          "name": "",
          "location": "",
          "description": "",
          "status": ""
        }}
      ]
    }}
    </OUTPUT>
    """
    result = perplexity_search(prompt)
    return {perplexity_search_key: result}


from tools.kakao_api_distance_tool import get_location_profile


def kakao_api_distance_tool(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    result = get_location_profile(target_area)
    return {kakao_api_distance_context_key: result}


def analysis_setting(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    total_units = start_input[total_units_key]
    main_type = start_input[main_type_key]
    gemini_search = state[gemini_search_key]
    kakao_api_distance_context = state[kakao_api_distance_context_key]
    perplexity_search = state[perplexity_search_key]

    system_prompt = PromptManager(PromptType.LOCATION_INSIGHT_SYSTEM).get_prompt()
    human_prompt = PromptManager(PromptType.LOCATION_INSIGHT_HUMAN).get_prompt(
        target_area=target_area,
        total_units=total_units,
        main_type=main_type,
        date=get_today_str(),
        gemini_search=gemini_search,
        kakao_api_distance_context=kakao_api_distance_context,
        perplexity_search=perplexity_search,
    )
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt),
    ]
    return {**state, messages_key: messages}


def agent(state: LocationInsightState) -> LocationInsightState:
    messages = state.get(messages_key, [])
    response = llm_with_tools.invoke(messages)
    new_messages = messages + [response]
    new_state = {**state, messages_key: new_messages}
    new_state[output_key] = {
        "result": response.content,
        gemini_search_key: state[gemini_search_key],
        kakao_api_distance_context_key: state[kakao_api_distance_context_key],
        perplexity_search_key: state[perplexity_search_key],
    }
    return new_state


def router(state: LocationInsightState):
    messages = state[messages_key]
    last_ai_message = messages[-1]
    if last_ai_message.tool_calls:
        return "tools"
    return "__end__"


web_search_key = "web_search"
analysis_setting_key = "analysis_setting"
tools_key = "tools"
agent_key = "agent"
gemini_search_key = "gemini_search"
kakao_api_distance_key = "kakao_api_distance"
perplexity_search_key = "perplexity_search"

graph_builder = StateGraph(LocationInsightState)

graph_builder.add_node(gemini_search_key, gemini_search_tool)
graph_builder.add_node(kakao_api_distance_key, kakao_api_distance_tool)
graph_builder.add_node(analysis_setting_key, analysis_setting)
graph_builder.add_node(perplexity_search_key, perplexity_search_tool)

graph_builder.add_node(tools_key, tool_node)
graph_builder.add_node(agent_key, agent)

graph_builder.add_edge(START, gemini_search_key)
graph_builder.add_edge(START, kakao_api_distance_key)
graph_builder.add_edge(START, perplexity_search_key)


graph_builder.add_edge(gemini_search_key, analysis_setting_key)
graph_builder.add_edge(kakao_api_distance_key, analysis_setting_key)
graph_builder.add_edge(perplexity_search_key, analysis_setting_key)
graph_builder.add_edge(analysis_setting_key, agent_key)

graph_builder.add_conditional_edges(agent_key, router, [tools_key, END])
graph_builder.add_edge(tools_key, agent_key)

location_insight_graph = graph_builder.compile()

In [1]:
invoke = await location_insight_graph.ainvoke(
    {
        "start_input": {
            "target_area": "서울 강남구 역삼동",
            "total_units": "1000세대",
            "main_type": "84제곱미터",
        }
    }
)

NameError: name 'location_insight_graph' is not defined

In [ ]:
print(invoke)

In [ ]:
print(invoke["location_insight_output"])

In [ ]:
print(invoke["gemini_search"])

In [ ]:
print(invoke["perplexity_search"])

In [3]:
from agents.analysis.location_insight_agent import location_insight_graph

invoke = await location_insight_graph.ainvoke(
    {
        "start_input": {
            "target_area": "서울 강남구 역삼동",
            "total_units": "1000세대",
            "main_type": "84제곱미터",
        }
    }
)

In [ ]:
print(invoke)

In [ ]:
output = invoke["location_insight_output"]

result_json = output["result"]

print("=" * 50)
print("📊 최종 분석 결과")
print("=" * 50)
print(result_json)

In [ ]:
print("=" * 50)
print("🔍 Gemini 검색 결과")
print("=" * 50)
print(output["gemini_search"])

In [ ]:
print("=" * 50)
print("📍 카카오 API 위치 정보")
print("=" * 50)
print(json.dumps(output["kakao_api_distance_context"], ensure_ascii=False, indent=2))

In [ ]:
print("=" * 50)
print("🌐 Perplexity 검색 결과")
print("=" * 50)
print(output["perplexity_search"])